In [13]:
import os 

In [14]:
%pwd

'/home/abdelilah/Documents/projet/textsummarizer/TextSummarizer'

In [15]:
%pwd





'/home/abdelilah/Documents/projet/textsummarizer/TextSummarizer'

In [16]:
from dataclasses import dataclass
from pathlib import Path

# return type
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path
    

In [17]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [18]:
from textSummarizer.logging import logger
class ConfigurationManager:
    def  __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        try:
            
            self.config = read_yaml(CONFIG_FILE_PATH)
        except FileNotFoundError as e:
            logger.error(f"Configuration file not found: {config_filepath}")
            raise e
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        return data_ingestion_config
    
    

In [19]:
import os 
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [20]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
        
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file,
            )
            logger.info(f"Downloaded {filename} - with following info: {headers}")
        else:
            logger.info(f"File {get_size(Path(self.config.local_data_file))} already exists.")
        
    def extract_zip_file(self):
        """
        zip_file_path:str
        Extracts the zip file into the data directory
        Functions retuens None
        
        """  
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted {self.config.local_data_file} to {unzip_path}")

In [21]:
# creating piplines
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2024-10-06 12:20:38,435: INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-10-06 12:20:38,438: INFO:common:yaml file: params.yaml loaded successfully]
[2024-10-06 12:20:38,439: INFO:common:Directory created: artifacts]
[2024-10-06 12:20:38,440: INFO:common:Directory created: artifacts/data_ingestion]
[2024-10-06 12:20:38,441: INFO:2773722855:File ~24223KB already exists.]
[2024-10-06 12:20:38,859: INFO:2773722855:Extracted artifacts/data_ingestion/data.zip to artifacts/data_ingestion]
